In [3]:
from raceplotly.plots import barplot
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.io as pio
pio.renderers.default = "notebook_connected"


In [4]:
Alex = pd.read_csv("AlexNet.csv")
Google = pd.read_csv("GoogleNet.csv")
Le = pd.read_csv("LeNet.csv")
Res = pd.read_csv("ResNet.csv")
Se = pd.read_csv("SeNet.csv")
VGG = pd.read_csv("VGG.csv")
Alex = Alex[Alex["Publication Year"] >= 2000].reset_index()
Google = Google[Google["Publication Year"] >= 2000].reset_index()
Le = Le[Le["Publication Year"] >= 2000].reset_index()
Res = Res[Res["Publication Year"] >= 2000].reset_index()
Se = Se[Se["Publication Year"] >= 2000].reset_index()
VGG = VGG[VGG["Publication Year"] >= 2000].reset_index()
df = pd.concat([Alex,Google,Le,Res,Se,VGG]).reset_index()

In [5]:
def change(x,new):
    if x in new:
        x = x.split(" ")[0]
    return x

In [6]:
# country_list = [df["Publisher Address"].str.rsplit(" ",1)[i][1] for i in range(len(df))]
from hashlib import new


df = df[df['Addresses'].notna()].reset_index(drop = True)
df_1 = df[["Addresses","Publication Year"]]
city_list = [df["Addresses"][i].rsplit(",", 2)[1] for i in range(len(df))]

In [38]:
df_city = pd.DataFrame(pd.value_counts(city_list))
a =  list(df_city.index)
#get the city with number
res = [a[i] for i in range(len(a)) if any(
    a[i][j].isdigit() for j in range(len(a[i])))]
#get the city with number in the front    
new = [res[i] for i in range(len(res)) if not any(
    res[i].split(" ")[1][j].isdigit() for j in range(len(res[i].split(" ")[1])))]
#get the city with number in the back
new_back = [res[i] for i in range(len(res)) if any(
    res[i].split(" ", 2)[1][j].isdigit() for j in range(len(res[i].split(" ")[1])))]
#get the city with number in the back
new_other =['Toulouse 9','Geneva 4','Paris 20','Paris 05','Amiens 1', 'Grenoble 9','Le Mans 9', 'Nice 3', 'Paris 13', 'Bratislava 15']


df_2 = df_1.copy()
df_2.loc[:,"Addresses"] = city_list
df_2["Addresses"] = df_2.Addresses.map(lambda x: x.split(" ")[1] if x in new else x)
df_2["Addresses"] = df_2.Addresses.map(lambda x: x.split(" ",2)[2] if x in new_back else x)
df_2["Addresses"] = df_2.Addresses.map(lambda x: 'Le Mans' if x == 'Le Mans 9' else x)
df_2["Addresses"] = df_2.Addresses.map(lambda x: x.split(" ")[0] if x in new_other else x)
df_2["Addresses"] = df_2.Addresses.map(lambda x: x.split(" ")[1] if x.split(" ")[0]=="" else x)
b = list(df_2.Addresses)
res_1 = [b[i] for i in range(len(b)) if any(
    b[i][j].isdigit() for j in range(len(b[i])))]

city_list_new = list(df_2["Addresses"])
df_city_new = pd.DataFrame(pd.value_counts(city_list_new))
df_city_new

,0
Beijing,1133
Shanghai,430
Seoul,297
Tamil,260
Jiangsu,192
...,...
Shirvan,1
RA Leiden,1
Kohat,1
Khyber,1


In [18]:
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

longitude = []
latitude = []

def findGeocode(country):
    try:
        geolocator = Nominatim(user_agent="your_app_name")

        return geolocator.geocode(country)

    except GeocoderTimedOut:

        return findGeocode(country)

for i in (df_city_new.index):

    if findGeocode(i) != None:

        loc = findGeocode(i)
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)

    else:
        latitude.append(np.nan)
        longitude.append(np.nan)


In [39]:
# now add this column to dataframe
location = pd.read_csv("Location_new.csv")
df_city_new["Longitude"] = longitude
df_city_new["Latitude"] = latitude
df_city_new= df_city_new.rename(columns = {0:"Magnitude"})
#df_city_new.to_csv("Location_new.csv")
df_city_new["Addresses"] = df_city_new.index
df_city_new

,Magnitude,Longitude,Latitude,Addresses
Beijing,1133,116.391276,39.906217,Beijing
Shanghai,430,121.469207,31.232276,Shanghai
Seoul,297,126.978291,37.566679,Seoul
Tamil,260,123.008770,8.425786,Tamil
Jiangsu,192,120.000000,33.000000,Jiangsu
...,...,...,...,...
Shirvan,1,57.930031,37.395352,Shirvan
RA Leiden,1,-68.913346,12.097171,RA Leiden
Kohat,1,71.439994,33.596714,Kohat
Khyber,1,74.796110,36.577770,Khyber


In [36]:
df_new = df_city_new.copy()
df_new["Addresses"] = df_new.Addresses.map(lambda x: x.split(" ")[1] if x.split(" ")[0]=="" else x)
df_city_new = pd.merge(df_new, df_city_new, on="Addresses", how="left")
df_city_new.head()


,Magnitude_x,Longitude_x,Latitude_x,Addresses,Magnitude_y,Longitude_y,Latitude_y
0,1133,116.391276,39.906217,Beijing,1133,116.391276,39.906217
1,430,121.469207,31.232276,Shanghai,430,121.469207,31.232276
2,297,126.978291,37.566679,Seoul,297,126.978291,37.566679
3,260,123.008770,8.425786,Tamil,260,123.008770,8.425786
4,192,120.000000,33.000000,Jiangsu,192,120.000000,33.000000


In [40]:
from tkinter.tix import TCL_DONT_WAIT
from pyecharts import options as opts
from pyecharts.charts import Geo, Timeline
from pyecharts.globals import SymbolType
from pyecharts.globals import ChartType
from pyecharts.charts import Tab


In [41]:

data_new = df_city_new
es = Geo().add_schema(maptype="world")
for i in range(len(data_new)):
    es.add_coordinate(
        data_new.index[i], data_new.Longitude[i], data_new.Latitude[i])
es.add(
    "",
    [list(z)
        for z in zip(list(data_new.index), list(data_new.Magnitude))],
    type_=ChartType.EFFECT_SCATTER,
    symbol_size=5,
)
es.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
es.set_global_opts(
    visualmap_opts=opts.VisualMapOpts(
        is_show=True,
        type_="color",
        range_color=['#f1e4ff', '#af78ff', '#7f4dd0', '#4d3077'],
        max_=250
    ),
    legend_opts=opts.LegendOpts(
        textstyle_opts=opts.TextStyleOpts(font_size=30, color='white')),
    title_opts=opts.TitleOpts(title="",
                                title_textstyle_opts=opts.TextStyleOpts(
                                    font_size=40),
                                subtitle_textstyle_opts=opts.TextStyleOpts(font_size=25)))


In [42]:
hm = Geo().add_schema(maptype="world")
for i in range(len(df_city_new)):
    hm.add_coordinate(
        df_city_new.index[i], df_city_new.Longitude[i], df_city_new.Latitude[i])
hm.add(
    "",
    [list(z) for z in zip(list(df_city_new.index), list(df_city_new.Magnitude))],
    type_=ChartType.HEATMAP,
    symbol_size=10,
)
hm.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
hm.set_global_opts(
    visualmap_opts=opts.VisualMapOpts(
        is_show=True,
        type_="color",
        range_color=['#f1e4ff', '#af78ff', '#7f4dd0', '#4d3077'],
        max_ = 250
    ),
    legend_opts=opts.LegendOpts(
        textstyle_opts=opts.TextStyleOpts(font_size=30, color='white')),
    title_opts=opts.TitleOpts(title="",
                              title_textstyle_opts=opts.TextStyleOpts(
                                    font_size=40),
                              subtitle_textstyle_opts=opts.TextStyleOpts(font_size=25)))


In [44]:
tab = Tab()
tab.add(es, "Effect Scatter")
tab.add(hm, "HeatMap")
tab.render("Map.html")


'c:\\Users\\Haowen\\OneDrive - Duke University\\Acadmic\\Resources\\STATS 401\\Final\\Map.html'

In [45]:
pd.read_csv("Location_new.csv")

,Unnamed: 0,Magnitude,Longitude,Latitude,Addresses
0,Beijing,1133,116.391276,39.906217,Beijing
1,Shanghai,430,121.469207,31.232276,Shanghai
2,Seoul,297,126.978291,37.566679,Seoul
3,Tamil,260,123.008770,8.425786,Tamil
4,Jiangsu,192,120.000000,33.000000,Jiangsu
...,...,...,...,...,...
1780,Shirvan,1,57.930031,37.395352,Shirvan
1781,RA Leiden,1,-68.913346,12.097171,RA Leiden
1782,Kohat,1,71.439994,33.596714,Kohat
1783,Khyber,1,74.796110,36.577770,Khyber
